In [ ]:
import os
import numpy as np
import pandas as pd

from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# sub 데이터 불러오기
sub = pd.read_csv('/content/drive/MyDrive/날씨 빅콘/data/220119(제출용)/2-2_검증데이터셋.csv', encoding = 'cp949')

In [ ]:
# 데이터형 변환
sub['yyyymmdd'] = pd.to_datetime(sub['yyyymmdd'],format="%Y-%m-%d")

In [ ]:
# 월, 일, 요일 생성
sub['month'] = sub['yyyymmdd'].dt.month
sub['day'] = sub['yyyymmdd'].dt.day
sub['요일'] = sub['yyyymmdd'].dt.weekday
sub['365일'] = sub['yyyymmdd'].dt.dayofyear

In [ ]:
# 계절 생성
conditionlist = [
    (sub['month'] >= 3) & (sub['month'] <= 5),
    (sub['month'] >= 6) & (sub['month'] <= 8),
    (sub['month'] >= 9) & (sub['month'] <= 11),
    (sub['month'] == 12) | (sub['month'] <= 2)]
choicelist = ['spring', 'summer', 'fall', 'winter']

sub['season'] = np.select(conditionlist, choicelist, default='NOT')

In [ ]:
sub

,yyyymmdd,area,sex,frequency,month,day,요일,365일,season
0,2016-01-01,강원,1,NaN,1,1,4,1,winter
1,2016-01-01,경기,1,NaN,1,1,4,1,winter
2,2016-01-01,경남,1,NaN,1,1,4,1,winter
3,2016-01-01,경북,1,NaN,1,1,4,1,winter
4,2016-01-01,광주,1,NaN,1,1,4,1,winter
...,...,...,...,...,...,...,...,...,...
12439,2016-12-31,전남,2,NaN,12,31,5,366,winter
12440,2016-12-31,전북,2,NaN,12,31,5,366,winter
12441,2016-12-31,제주,2,NaN,12,31,5,366,winter
12442,2016-12-31,충남,2,NaN,12,31,5,366,winter


In [ ]:
# train 데이터 처리
hospital = pd.read_csv('/content/drive/MyDrive/날씨 빅콘/data/back_hospital.csv')
hospital = hospital.drop(['Unnamed: 0'],axis=1)
hospital.columns = ['yyyymmdd','area','sex','frequency']

In [ ]:
hospital['yyyymmdd'] = pd.to_datetime(hospital['yyyymmdd'],format="%Y%m%d")
hospital['month'] = hospital['yyyymmdd'].dt.month
hospital['day'] = hospital['yyyymmdd'].dt.day
hospital['요일'] = hospital['yyyymmdd'].dt.weekday
hospital['365일'] = hospital['yyyymmdd'].dt.dayofyear

In [ ]:
conditionlist = [
    (hospital['month'] >= 3) & (hospital['month'] <= 5),
    (hospital['month'] >= 6) & (hospital['month'] <= 8),
    (hospital['month'] >= 9) & (hospital['month'] <= 11),
    (hospital['month'] == 12) | (hospital['month'] <= 2)]
choicelist = ['spring', 'summer', 'fall', 'winter']

hospital['season'] = np.select(conditionlist, choicelist, default='NOT')

In [ ]:
hospital

,yyyymmdd,area,sex,frequency,month,day,요일,365일,season
0,2012-01-01,강원,1,3,1,1,6,1,winter
1,2012-01-01,강원,2,3,1,1,6,1,winter
2,2012-01-01,경기,1,4,1,1,6,1,winter
3,2012-01-01,경기,2,5,1,1,6,1,winter
4,2012-01-01,경남,1,2,1,1,6,1,winter
...,...,...,...,...,...,...,...,...,...
49669,2015-12-31,제주,2,0,12,31,3,365,winter
49670,2015-12-31,충남,1,0,12,31,3,365,winter
49671,2015-12-31,충남,2,0,12,31,3,365,winter
49672,2015-12-31,충북,1,1,12,31,3,365,winter


# 지역별,일(365)별 평균

In [ ]:
area_365days = hospital.groupby(['area','365일'])['frequency'].mean()
area_365days = pd.DataFrame(area_365days)

In [ ]:
area_365days

frequency
area 365일           
강원   1         3.250
     2         1.250
     3         1.000
     4         0.875
     5         1.375
...              ...
충북   362       0.625
     363       0.750
     364       1.000
     365       0.375
     366       1.500

[6222 rows x 1 columns]

In [ ]:
area_365days = area_365days.reset_index()
area_365days

,area,365일,frequency
0,강원,1,3.250
1,강원,2,1.250
2,강원,3,1.000
3,강원,4,0.875
4,강원,5,1.375
...,...,...,...
6217,충북,362,0.625
6218,충북,363,0.750
6219,충북,364,1.000
6220,충북,365,0.375


In [ ]:
area_365days_sub = pd.merge(sub, area_365days, how="left", on=['area','365일'])

In [ ]:
area_365days_sub

,yyyymmdd,area,sex,frequency_x,month,day,요일,365일,season,frequency_y
0,2016-01-01,강원,1,NaN,1,1,4,1,winter,3.250
1,2016-01-01,경기,1,NaN,1,1,4,1,winter,5.750
2,2016-01-01,경남,1,NaN,1,1,4,1,winter,3.125
3,2016-01-01,경북,1,NaN,1,1,4,1,winter,3.750
4,2016-01-01,광주,1,NaN,1,1,4,1,winter,2.375
...,...,...,...,...,...,...,...,...,...,...
12439,2016-12-31,전남,2,NaN,12,31,5,366,winter,2.000
12440,2016-12-31,전북,2,NaN,12,31,5,366,winter,1.000
12441,2016-12-31,제주,2,NaN,12,31,5,366,winter,1.000
12442,2016-12-31,충남,2,NaN,12,31,5,366,winter,1.000


In [ ]:
area_365days_sub['frequency_x'] = area_365days_sub['frequency_y']

In [ ]:
area_365days_sub = area_365days_sub[['yyyymmdd','area','sex','frequency_x']]

In [ ]:
area_365days_sub.columns = ['yyyymmdd','area','sex','frequency']

In [ ]:
area_365days_sub

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,3.250
1,2016-01-01,경기,1,5.750
2,2016-01-01,경남,1,3.125
3,2016-01-01,경북,1,3.750
4,2016-01-01,광주,1,2.375
...,...,...,...,...
12439,2016-12-31,전남,2,2.000
12440,2016-12-31,전북,2,1.000
12441,2016-12-31,제주,2,1.000
12442,2016-12-31,충남,2,1.000


In [ ]:
area_365days_sub.to_csv("/content/drive/MyDrive/날씨 빅콘/data/220119(제출용)/.../area_365일_평균.csv",index=False)

# 지역별, 일별 평균 (1위)

In [ ]:
area_days = hospital.groupby(['area','day'])['frequency'].mean()
area_days = pd.DataFrame(area_days)

In [ ]:
area_days

frequency
area day           
강원   1     2.333333
     2     1.041667
     3     1.020833
     4     1.052083
     5     0.906250
...             ...
충북   27    0.989583
     28    0.916667
     29    1.200000
     30    1.068182
     31    0.785714

[527 rows x 1 columns]

In [ ]:
area_days = area_days.reset_index()
area_days

,area,day,frequency
0,강원,1,2.333333
1,강원,2,1.041667
2,강원,3,1.020833
3,강원,4,1.052083
4,강원,5,0.906250
...,...,...,...
522,충북,27,0.989583
523,충북,28,0.916667
524,충북,29,1.200000
525,충북,30,1.068182


In [ ]:
area_days_sub = pd.merge(sub, area_days, how="left", on=['area','day'])

In [ ]:
area_days_sub

,yyyymmdd,area,sex,frequency_x,month,day,요일,365일,season,frequency_y
0,2016-01-01,강원,1,NaN,1,1,4,1,winter,2.333333
1,2016-01-01,경기,1,NaN,1,1,4,1,winter,5.937500
2,2016-01-01,경남,1,NaN,1,1,4,1,winter,3.302083
3,2016-01-01,경북,1,NaN,1,1,4,1,winter,4.000000
4,2016-01-01,광주,1,NaN,1,1,4,1,winter,1.781250
...,...,...,...,...,...,...,...,...,...,...
12439,2016-12-31,전남,2,NaN,12,31,5,366,winter,1.660714
12440,2016-12-31,전북,2,NaN,12,31,5,366,winter,1.125000
12441,2016-12-31,제주,2,NaN,12,31,5,366,winter,0.267857
12442,2016-12-31,충남,2,NaN,12,31,5,366,winter,0.982143


In [ ]:
area_days_sub['frequency_x'] = area_days_sub['frequency_y']

In [ ]:
area_days_sub = area_days_sub[['yyyymmdd','area','sex','frequency_x']]
area_days_sub.columns = ['yyyymmdd','area','sex','frequency']

In [ ]:
area_days_sub

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,2.333333
1,2016-01-01,경기,1,5.937500
2,2016-01-01,경남,1,3.302083
3,2016-01-01,경북,1,4.000000
4,2016-01-01,광주,1,1.781250
...,...,...,...,...
12439,2016-12-31,전남,2,1.660714
12440,2016-12-31,전북,2,1.125000
12441,2016-12-31,제주,2,0.267857
12442,2016-12-31,충남,2,0.982143


In [ ]:
area_days_sub.to_csv("/content/drive/MyDrive/날씨 빅콘/data/220119(제출용)/.../area_day_평균.csv",index=False)

# 지역별 평균

In [ ]:
area = hospital.groupby(['area'])['frequency'].mean()
area = pd.DataFrame(area)

In [ ]:
area

,frequency
area,
강원,1.050308
경기,4.540383
경남,2.046201
경북,1.921287
광주,0.726899
대구,1.150924
대전,0.664956
부산,1.761123
서울,3.543463


In [ ]:
area = area.reset_index()
area

,area,frequency
0,강원,1.050308
1,경기,4.540383
2,경남,2.046201
3,경북,1.921287
4,광주,0.726899
5,대구,1.150924
6,대전,0.664956
7,부산,1.761123
8,서울,3.543463
9,세종,0.055784


In [ ]:
area_sub = pd.merge(sub, area, how="left", on=['area'])

In [ ]:
area_sub

,yyyymmdd,area,sex,frequency_x,month,day,요일,365일,season,frequency_y
0,2016-01-01,강원,1,NaN,1,1,4,1,winter,1.050308
1,2016-01-01,경기,1,NaN,1,1,4,1,winter,4.540383
2,2016-01-01,경남,1,NaN,1,1,4,1,winter,2.046201
3,2016-01-01,경북,1,NaN,1,1,4,1,winter,1.921287
4,2016-01-01,광주,1,NaN,1,1,4,1,winter,0.726899
...,...,...,...,...,...,...,...,...,...,...
12439,2016-12-31,전남,2,NaN,12,31,5,366,winter,1.550992
12440,2016-12-31,전북,2,NaN,12,31,5,366,winter,1.303217
12441,2016-12-31,제주,2,NaN,12,31,5,366,winter,0.272074
12442,2016-12-31,충남,2,NaN,12,31,5,366,winter,1.093087


In [ ]:
area_sub['frequency_x'] = area_sub['frequency_y']

In [ ]:
area_sub = area_sub[['yyyymmdd','area','sex','frequency_x']]
area_sub.columns = ['yyyymmdd','area','sex','frequency']

In [ ]:
area_sub

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,1.050308
1,2016-01-01,경기,1,4.540383
2,2016-01-01,경남,1,2.046201
3,2016-01-01,경북,1,1.921287
4,2016-01-01,광주,1,0.726899
...,...,...,...,...
12439,2016-12-31,전남,2,1.550992
12440,2016-12-31,전북,2,1.303217
12441,2016-12-31,제주,2,0.272074
12442,2016-12-31,충남,2,1.093087


In [ ]:
area_sub.to_csv("/content/drive/MyDrive/날씨 빅콘/data/220119(제출용)/.../area_평균.csv",index=False)

# 지역별, 성별 평균

In [ ]:
area_sex = hospital.groupby(['area','sex'])['frequency'].mean()
area_sex = pd.DataFrame(area_sex)

In [ ]:
area_sex

frequency
area sex           
강원   1     1.120465
     2     0.980151
경기   1     4.626968
     2     4.453799
경남   1     1.989049
     2     2.103354
경북   1     1.899384
     2     1.943190
광주   1     0.652977
     2     0.800821
대구   1     1.138261
     2     1.163587
대전   1     0.690623
     2     0.639288
부산   1     1.889117
     2     1.633128
서울   1     3.650240
     2     3.436687
세종   1     0.063655
     2     0.047912
울산   1     0.466804
     2     0.464066
인천   1     1.073922
     2     1.005476
전남   1     1.431896
     2     1.670089
전북   1     1.169747
     2     1.436687
제주   1     0.281999
     2     0.262149
충남   1     1.114305
     2     1.071869
충북   1     1.014374
     2     0.967830

In [ ]:
area_sex = area_sex.reset_index()
area_sex

,area,sex,frequency
0,강원,1,1.120465
1,강원,2,0.980151
2,경기,1,4.626968
3,경기,2,4.453799
4,경남,1,1.989049
5,경남,2,2.103354
6,경북,1,1.899384
7,경북,2,1.943190
8,광주,1,0.652977
9,광주,2,0.800821


In [ ]:
area_sex_sub = pd.merge(sub, area_sex, how="left", on=['area','sex'])

In [ ]:
area_sex_sub

,yyyymmdd,area,sex,frequency_x,month,day,요일,365일,season,frequency_y
0,2016-01-01,강원,1,NaN,1,1,4,1,winter,1.120465
1,2016-01-01,경기,1,NaN,1,1,4,1,winter,4.626968
2,2016-01-01,경남,1,NaN,1,1,4,1,winter,1.989049
3,2016-01-01,경북,1,NaN,1,1,4,1,winter,1.899384
4,2016-01-01,광주,1,NaN,1,1,4,1,winter,0.652977
...,...,...,...,...,...,...,...,...,...,...
12439,2016-12-31,전남,2,NaN,12,31,5,366,winter,1.670089
12440,2016-12-31,전북,2,NaN,12,31,5,366,winter,1.436687
12441,2016-12-31,제주,2,NaN,12,31,5,366,winter,0.262149
12442,2016-12-31,충남,2,NaN,12,31,5,366,winter,1.071869


In [ ]:
area_sex_sub['frequency_x'] = area_sex_sub['frequency_y']

In [ ]:
area_sex_sub = area_sex_sub[['yyyymmdd','area','sex','frequency_x']]
area_sex_sub.columns = ['yyyymmdd','area','sex','frequency']

In [ ]:
area_sex_sub

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,1.120465
1,2016-01-01,경기,1,4.626968
2,2016-01-01,경남,1,1.989049
3,2016-01-01,경북,1,1.899384
4,2016-01-01,광주,1,0.652977
...,...,...,...,...
12439,2016-12-31,전남,2,1.670089
12440,2016-12-31,전북,2,1.436687
12441,2016-12-31,제주,2,0.262149
12442,2016-12-31,충남,2,1.071869


In [ ]:
area_sex_sub.to_csv("/content/drive/MyDrive/날씨 빅콘/data/220119(제출용)/.../area_sex_평균.csv",index=False)

# 지역별, 일별, 성별 평균

In [ ]:
area_day_sex = hospital.groupby(['area','day','sex'])['frequency'].mean()
area_day_sex = pd.DataFrame(area_day_sex)

In [ ]:
area_day_sex

frequency
area day sex           
강원   1   1     3.250000
         2     1.416667
     2   1     1.083333
         2     1.000000
     3   1     1.000000
...                 ...
충북   29  2     1.266667
     30  1     0.886364
         2     1.250000
     31  1     0.678571
         2     0.892857

[1054 rows x 1 columns]

In [ ]:
area_day_sex = area_day_sex.reset_index()
area_day_sex

,area,day,sex,frequency
0,강원,1,1,3.250000
1,강원,1,2,1.416667
2,강원,2,1,1.083333
3,강원,2,2,1.000000
4,강원,3,1,1.000000
...,...,...,...,...
1049,충북,29,2,1.266667
1050,충북,30,1,0.886364
1051,충북,30,2,1.250000
1052,충북,31,1,0.678571


In [ ]:
area_day_sex_sub = pd.merge(sub, area_day_sex, how="left", on=['area','day','sex'])

In [ ]:
area_day_sex_sub

,yyyymmdd,area,sex,frequency_x,month,day,요일,365일,season,frequency_y
0,2016-01-01,강원,1,NaN,1,1,4,1,winter,3.250000
1,2016-01-01,경기,1,NaN,1,1,4,1,winter,5.916667
2,2016-01-01,경남,1,NaN,1,1,4,1,winter,3.166667
3,2016-01-01,경북,1,NaN,1,1,4,1,winter,4.854167
4,2016-01-01,광주,1,NaN,1,1,4,1,winter,1.458333
...,...,...,...,...,...,...,...,...,...,...
12439,2016-12-31,전남,2,NaN,12,31,5,366,winter,2.000000
12440,2016-12-31,전북,2,NaN,12,31,5,366,winter,1.142857
12441,2016-12-31,제주,2,NaN,12,31,5,366,winter,0.250000
12442,2016-12-31,충남,2,NaN,12,31,5,366,winter,1.214286


In [ ]:
area_day_sex_sub['frequency_x'] = area_day_sex_sub['frequency_y']

In [ ]:
area_day_sex_sub = area_day_sex_sub[['yyyymmdd','area','sex','frequency_x']]
area_day_sex_sub.columns = ['yyyymmdd','area','sex','frequency']

In [ ]:
area_day_sex_sub

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,3.250000
1,2016-01-01,경기,1,5.916667
2,2016-01-01,경남,1,3.166667
3,2016-01-01,경북,1,4.854167
4,2016-01-01,광주,1,1.458333
...,...,...,...,...
12439,2016-12-31,전남,2,2.000000
12440,2016-12-31,전북,2,1.142857
12441,2016-12-31,제주,2,0.250000
12442,2016-12-31,충남,2,1.214286


In [ ]:
area_day_sex_sub.to_csv("/content/drive/MyDrive/날씨 빅콘/data/220119(제출용)/.../area_day_sex_평균.csv",index=False)